In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

/home/emreunel/anaconda3/envs/ass3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence
/home/emreunel/anaconda3/envs/ass3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, namedtuple, defaultdict, Sequence
/home/emreunel/anaconda3/envs/ass3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

In [3]:
data = pd.read_csv("dataset/preprocessed.csv")
data.shape

(762106, 44)

In [4]:
data = data.drop(data[data.target == -1].index)
data.shape

(762094, 44)

In [5]:
# Separate input features and target
targets = data.target
targets.head()

0    3
1    5
2    2
3    2
4    1
Name: target, dtype: int64

In [6]:
targets -= 1
targets.head()

0    2
1    4
2    1
3    1
4    0
Name: target, dtype: int64

In [7]:
targets = targets.to_numpy()

In [8]:
features = data.drop('target', axis=1)

In [9]:
features = features.to_numpy()

In [10]:
X_trainAndVal, X_test, y_trainAndVal, y_test = train_test_split(features, targets, test_size = 0.2, random_state = 0)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_trainAndVal, y_trainAndVal, test_size = 0.2, random_state = 0)

In [12]:
X_train.shape



(487740, 43)

In [14]:
unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[     0      1      2      3      4]
 [ 50166  55949  87609 117459 176557]]


In [15]:
sm = SMOTETomek(random_state = 27, n_jobs = -1)

X_train, y_train = sm.fit_sample(X_train, y_train)

/home/emreunel/anaconda3/envs/ass3/lib/python3.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'k' is deprecated from 0.2 and will be removed in 0.4. Use 'k_neighbors' instead.
  category=DeprecationWarning)
/home/emreunel/anaconda3/envs/ass3/lib/python3.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'm' is deprecated from 0.2 and will be removed in 0.4. Use 'm_neighbors' instead.
  category=DeprecationWarning)


In [16]:
unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[     0      1      2      3      4]
 [162635 155765 149211 144863 147975]]


In [17]:
X_train.shape


(760449, 43)

In [18]:
first_feature_count =           22
last_feature_count =            21


In [19]:
print((X_train ).shape)
print(( y_train).shape)


(760449, 43)
(760449,)


In [20]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

/home/emreunel/anaconda3/envs/ass3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [21]:
batch_size = 1000
epochs = 50
input_dim = first_feature_count + last_feature_count 
output_dim = 5
lr_rate = 0.001


In [22]:
class datasetLoad(Dataset):
    def __init__(self, features,labels):
        self.features = features
        self.labels = labels
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    


In [23]:
print(type(X_train))
X_train = datasetLoad(X_train, y_train)
X_val = datasetLoad(X_val, y_val)
X_test = datasetLoad(X_test, y_test)


<class 'numpy.ndarray'>


In [24]:
print(type(X_train))
train_loader = torch.utils.data.DataLoader(dataset = X_train, batch_size = batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = X_val, batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = X_test, batch_size = batch_size, shuffle=True)

<class '__main__.datasetLoad'>


In [25]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs
    
    

In [26]:
model = LogisticRegression(input_dim, output_dim)

In [27]:
criterion = torch.nn.CrossEntropyLoss()


In [28]:
# sgd ya da adam kullanilabilir
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [29]:
print(len(X_train))

760449


In [30]:
import datetime
a = datetime.datetime.now().replace(microsecond=0)
for epoch in range(epochs):
    for i, (train_feature, train_label) in enumerate(train_loader):
        train_feature = Variable(train_feature)
        train_label = Variable(train_label)

        optimizer.zero_grad()
        outputs = model(train_feature.float())
        loss = criterion(outputs.float(), train_label)
        loss.backward()
        optimizer.step()

        
        if i % (len(X_train)// (4 * batch_size))  == 0:
            # calculate Accuracy
            val_correct = 0
            val_total = 0
            for val_features, val_labels in val_loader:
                val_features = Variable(val_features)
                with torch.no_grad():    
                    val_outputs = model(val_features.float())
                _, val_predicted = torch.max(val_outputs.data, 1)
                val_total+= val_labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                val_correct+= (val_predicted == val_labels).sum()
            val_accuracy = 100 * val_correct / val_total
            # if(epoch % 5 == 0 and i == 0):
            #     print("val Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch, i, loss.item(), val_accuracy))
            print("val Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch, i, loss.item(), val_accuracy))
             


val Epoch: 0. Iteration: 0. Loss: 1.7577402591705322. Accuracy: 19.
val Epoch: 0. Iteration: 190. Loss: 1.7069194316864014. Accuracy: 22.
val Epoch: 0. Iteration: 380. Loss: 1.606073260307312. Accuracy: 25.
val Epoch: 0. Iteration: 570. Loss: 1.5699493885040283. Accuracy: 28.
val Epoch: 0. Iteration: 760. Loss: 1.5322269201278687. Accuracy: 29.
val Epoch: 1. Iteration: 0. Loss: 1.5024399757385254. Accuracy: 29.
val Epoch: 1. Iteration: 190. Loss: 1.5240098237991333. Accuracy: 29.
val Epoch: 1. Iteration: 380. Loss: 1.4797742366790771. Accuracy: 30.
val Epoch: 1. Iteration: 570. Loss: 1.47549307346344. Accuracy: 30.
val Epoch: 1. Iteration: 760. Loss: 1.4083991050720215. Accuracy: 30.
val Epoch: 2. Iteration: 0. Loss: 1.4524184465408325. Accuracy: 30.
val Epoch: 2. Iteration: 190. Loss: 1.4410929679870605. Accuracy: 30.
val Epoch: 2. Iteration: 380. Loss: 1.4753949642181396. Accuracy: 31.
val Epoch: 2. Iteration: 570. Loss: 1.4333246946334839. Accuracy: 31.
val Epoch: 2. Iteration: 760.

In [31]:
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

0:07:22
